In [1]:
from nltk.corpus import stopwords
import string
import re
import os
from collections import Counter

rootdir_pos = '/Users/Mal/Desktop/review_polarity/txt_sentoken/pos'
rootdir_neg = '/Users/Mal/Desktop/review_polarity/txt_sentoken/neg'

In [2]:
def load_doc(filename):
    file = open(filename, "r")
    text = file.read()
    file.close()
    return text

def clean_doc(doc):
    tokens = doc.split()
    re_punc = re.compile("[%s]" % re.escape(string.punctuation))
    tokens = [re_punc.sub("", w) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words("english"))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

def add_doc_to_vocab(filename, vocab):
    doc = load_doc(filename)
    tokens = clean_doc(doc)
    vocab.update(tokens)

In [3]:
vocab = Counter()

def process_docs_tokens(pos, neg):
    for subdir, dirs, files in os.walk(pos):
        for file in files:
            filepath = subdir + os.sep + file
            #print(file)
            if file.startswith("cv9"):
                #print("here", file)
                continue
            else:
                add_doc_to_vocab(filepath, vocab)

    for subdir, dirs, files in os.walk(neg):
        for file in files:
            filepath = subdir + os.sep + file
            #print(file)
            if file.startswith("cv9"):
                #print("here", file)
                continue
            else:
                add_doc_to_vocab(filepath, vocab)

    #print(len(vocab))
    #print(vocab.most_common(50))

def save_list(lines, filename):
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

process_docs_tokens(rootdir_pos, rootdir_neg)
min_occurrence = 2
tokens = [k for k,c in vocab.items() if c >= min_occurrence]
print(len(tokens))
save_list(tokens,"vocab.txt")

25767


In [4]:
def doc_to_line(filename, vocab):
    doc = load_doc(filename)
    tokens = clean_doc(doc)
    tokens = [w for w in tokens if w in vocab]
    return  " ".join(tokens)

def process_docs(directory, vocab, is_train):
    lines = list()
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            filepath = subdir + os.sep + file
            #print(file)
            if is_train and file.startswith("cv9"):
                #print("here", file)
                continue
            if not is_train and not file.startswith("cv9"):
                continue
            else:
                add_doc_to_vocab(filepath, vocab)
                line = doc_to_line(filepath, vocab)
                lines.append(line)
    return lines

def load_clean_dataset(vocab, is_train):
    neg = process_docs(rootdir_neg , vocab, is_train)
    pos = process_docs(rootdir_pos, vocab, is_train)
    docs = neg + pos
    labels = [0 for _ in range(len(neg))] + [1 for _ in range(len(pos))]
    return docs, labels

from keras.preprocessing.text import Tokenizer

def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer


Using TensorFlow backend.


In [10]:

from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
def define_model(n_words):
  # define network
    model = Sequential()
    model.add(Dense(50, input_shape=(n_words,), activation= 'relu' ))
    model.add(Dense(1, activation= 'sigmoid' ))
  # compile network
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy'])
  # summarize defined model
    model.summary()
    #plot_model(model, to_file='model.png' , show_shapes=True)
    return model

vocab_filename = "vocab.txt"
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

train_docs, ytrain = load_clean_dataset(vocab, True)
test_docs, ytest = load_clean_dataset(vocab, False)
tokenizer = create_tokenizer(train_docs)
#print(Xtrain.shape, Xtest.shape)
Xtrain = tokenizer.texts_to_matrix(train_docs, mode="binary")
Xtest = tokenizer.texts_to_matrix(test_docs, mode="binary")
# define the model
n_words = Xtest.shape[1]
model = define_model(n_words)
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print( 'Test Accuracy: %f' % (acc*100))


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 50)                2213900   
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 51        
Total params: 2,213,951
Trainable params: 2,213,951
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 - 5s - loss: 0.4532 - accuracy: 0.8000
Epoch 2/10
 - 4s - loss: 0.0376 - accuracy: 0.9978
Epoch 3/10
 - 4s - loss: 0.0110 - accuracy: 1.0000
Epoch 4/10
 - 4s - loss: 0.0057 - accuracy: 1.0000
Epoch 5/10
 - 4s - loss: 0.0032 - accuracy: 1.0000
Epoch 6/10
 - 4s - loss: 0.0018 - accuracy: 1.0000
Epoch 7/10
 - 4s - loss: 0.0010 - accuracy: 1.0000
Epoch 8/10
 - 4s - loss: 6.7449e-04 - accuracy: 1.0000
Epoch 9/10
 - 4s - loss: 4.7122e-04 - accuracy: 1.0000
Epoch 10/10
 - 4s - loss: 3.4647e-04 - 

In [11]:
def predict_sentiment(review, vocab, tokenizer, model):
    tokens = clean_doc(review)
    tokens = [w for w in tokens if w in vocab]
    line = ' '.join(tokens)
    encoded = tokenizer.texts_to_matrix([line], mode= 'binary' )
    yhat = model.predict(encoded, verbose=0)
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos,  'POSITIVE'

In [12]:
text = 'Best movie ever! It was great, I recommend it.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))
text = 'This is a bad movie.'
percent, sentiment = predict_sentiment(text, vocab, tokenizer, model)
print('Review: [%s]\nSentiment: %s (%.3f%%)' % (text, sentiment, percent*100))


Review: [Best movie ever! It was great, I recommend it.]
Sentiment: POSITIVE (56.387%)
Review: [This is a bad movie.]
Sentiment: NEGATIVE (64.274%)
